In [6]:
""" goal: analysis of kineamtics of trials.
align multiople trials of same task.
assess statistics of actions

"""

%load_ext autoreload
%autoreload 2

from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *


animal = "Pancho";
date = 200226
expt = "pilot"
session = 1
# animal = "Pancho";
# date = 200226
# expt = "pilot"
# session = 3

from tools.preprocess import *
filedata = loadSingleData(animal, date, expt, session, resave_as_dict=False, load_resaved_data=True, 
                      resave_overwrite=False)

from tools.utils import *
getMultTrialsTaskStages(filedata)

# --- save dir
SAVEDIR = plotMakeSaveDir(filedata, "analysis_twoline_planning_041020")
                          
    
targ = {
    "task_stage":["two_lines_anywhere"],
    "behtype":["Trace (instant)"],
    "fracsuccess_min":[0.25]
}

trials_list = getIndsTrials(filedata, targ)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
found one matching filename: ['/data2/animals/Pancho/200226/200226_121930_pilot_Pancho_1.h5']
found one matching filename: ['/data2/animals/Pancho/200226/200226_121930_pilot_Pancho_1.pkl']
-- loaded presaved data: /data2/animals/Pancho/200226/200226_121930_pilot_Pancho_1.pkl
got 617 total trials
got 159 total trials


In [12]:
getTrialsUniqueTaskname(filedata, 10)

'3-onedot_2'

In [16]:
for t in trials_list:
    print(t)

363
364
366
368
369
370
372
373
375
377
378
380
381
382
383
384
385
386
393
394
395
397
398
399
400
401
402
404
405
406
407
409
410
411
412
413
415
416
437
438
439
440
441
442
443
444
445
446
447
448
450
451
452
453
454
455
457
458
459
460
461
462
463
464
465
466
467
468
470
472
474
475
476
477
478
479
498
499
500
502
503
504
506
507
508
510
511
512
513
514
515
516
517
518
520
521
522
523
525
527
529
532
533
537
538
539
541
542
543
544
545
547
549
551
552
553
554
555
556
557
558
559
560
562
563
564
565
566
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
584
585
586
587
588
590
592
593
594
595
607
608
609
612
613
616


In [38]:

def removeRedundantTrials(filedata, trials_list):
    """
    NOTE: currently works by removing trials that are followed by trial in
    one ind. i.e, if there are a sequence of trials, 565 566 567, that are all the
    same block and same taskname, then will only keep 567. the output
    trials_list will have those trials removed
    - must resort to this since a given block can repeat, and so even with unique block and 
    task name is not sure if this is really a unique task.
    
    """
    tasknamedict = {}
    method = "keep_latest"

    for t in trials_list:
        taskname = getTrialsUniqueTaskname(filedata, t)

        # -- keep track of tasks
        if taskname in tasknamedict.keys():
            tasknamedict[taskname].append(t)
        else:
            tasknamedict[taskname]=[t]

    # -- if multiple, keep one
    trials_to_remove = []
    for task, trials in tasknamedict.items():
        if method=="keep_latest":
#             print(trials)
#             print((np.diff(trials)==1).nonzero())
            tmp = (np.diff(trials)==1).nonzero()[0]
    #         trials[tmp]
            trials_to_remove.extend([trials[int(t.reshape(-1,))] for t in tmp]) # inds that are followed by ind+1. those are removed.
    #         for t in tmp:
    #             trials_to_remove

    #         trials_to_keep.append(max(trials))
        else:
            assert False, "not coded"

    trials_list = [t for t in trials_list if t not in trials_to_remove]
    
    return trials_list


trials_list = removeRedundantTrials(filedata, trials_list)

[363, 465, 564, 565]
(array([2]),)
[364, 366, 521, 522, 523, 608]
(array([2, 3]),)
[368, 369, 513, 514, 515, 559, 560, 562]
(array([0, 2, 3, 5]),)
[370, 372, 373, 541, 542, 558]
(array([1, 3]),)
[375, 377, 378, 520, 609, 612]
(array([1]),)
[380, 381, 500, 502, 566]
(array([0]),)
[382, 383, 384, 503, 504, 506, 584, 585, 586]
(array([0, 1, 3, 6, 7]),)
[385, 386, 476, 556]
(array([0]),)
[393, 394, 395, 467, 468, 470, 575, 576]
(array([0, 1, 3, 6]),)
[397, 398, 525, 581, 582]
(array([0, 3]),)
[399, 400, 401, 510, 511, 512, 578, 579, 580]
(array([0, 1, 3, 4, 6, 7]),)
[402, 474, 475, 554, 555]
(array([1, 3]),)
[404, 459, 460, 547]
(array([1]),)
[405, 406, 516, 517, 518, 568]
(array([0, 2, 3]),)
[407, 409, 410, 532, 533, 537, 587, 588]
(array([1, 3, 6]),)
[411, 412, 413, 527, 529, 543, 544, 545]
(array([0, 1, 5, 6]),)
[415, 416, 437, 461, 462, 552, 553, 613, 616]
(array([0, 3, 5]),)
[438, 439, 440, 477, 478, 479, 594, 595, 607]
(array([0, 1, 3, 4, 6]),)
[441, 442, 443, 507, 572, 573, 574]
(ar

In [ ]:
    
[getTrialsBlockParams(filedata, t) for t in [395, 576]]



def test():
    # note, this was tring to get actual block progressino numbers, but this just gives me things
    # identical to block num...
    # 
    trials_within_block = [filedata["trials"][t]["TrialWithinBlock"][0][0] for t in getIndsTrials(filedata)]
    
    first_trials = (np.diff(np.array(trials_within_block))<0).nonzero()[0]+1
    first_trials = np.concatenate([np.array(0).reshape(-1,), first_trials, np.array(len(trials_within_block)).reshape(-1,)])

    out = []
    c=1
    for on, off in zip(first_trials[:-1], first_trials[1:]):
        out.extend([c for _ in range(off-on)])
        c+=1
    return out


# invec = [1,2,3,1,2,1,2,3]
test()

tmp = []

for t in getIndsTrials(filedata):
    tmp.append(filedata["trials"][t]["TrialWithinBlock"][0][0])
plt.figure()

inds = np.diff(tmp)<0
plt.plot(tmp, '-k')

In [ ]:
tl = [363, 465, 565]
plotMultTrialsSimple(filedata, trials_list)